In [75]:
import pandas as pd
import numpy as np
import ipywidgets as ipw
import bql
import bqwidgets as bqw
from bqplot import pyplot as plt
import bqviz
import bqplot as bqp
from ipywidgets import widgets
import plotly.graph_objs as go
import plotly.express as px
import plotly
import datetime
from plotly.subplots import make_subplots


In [76]:
bq = bql.Service()

In [77]:
def bizd_px():
    bql_item = bq.func.dropna(bq.data.px_last(dates=bq.func.range('-5y', '0d')))
    bql_universe = 'BIZD US Equity'
    bql_request = bql.Request(bql_universe, {'price': bql_item})
    bql_response = bq.execute(bql_request)
    px = pd.DataFrame(bql_response.single().df()["price"])
    date = pd.DataFrame(bql_response.single().df()["DATE"])
    df  = pd.concat([date, px], axis = 1).reset_index(drop=True)
    df.set_index("DATE", inplace = True)
    return df

In [78]:
def bizd_dvd():
    dps = bq.func.dropna(bq.data.dividend_per_share_12m(dates=bq.func.range('-5y', '0d')))
    bql_universe = 'BIZD US Equity'
    bql_request = bql.Request(bql_universe, {'dps_12m':dps})
    bql_response = bq.execute(bql_request)
    dps = bql_response.single().df()["dps_12m"]
    date = bql_response.single().df()["DATE"]
    df  = pd.concat([date, dps], axis = 1).reset_index(drop=True)
    df.set_index("DATE", inplace = True)
    return df

In [79]:
bizd_df = bizd_px().merge(bizd_dvd(), how = "inner",left_index = True, right_index = True)
bizd_df["ytm"] = bizd_df["dps_12m"]/bizd_df["price"]
bizd_df.pop("dps_12m")
multi_bizd = 100 / bizd_df["price"][0]
bizd_df["price"] = bizd_df["price"]*multi_bizd

In [80]:
def xmpt_px():
    bql_item = bq.func.dropna(bq.data.px_last(dates=bq.func.range('-5y', '0d')))
    bql_universe = 'XMPT US Equity'
    bql_request = bql.Request(bql_universe, {'price': bql_item})
    bql_response = bq.execute(bql_request)
    px = pd.DataFrame(bql_response.single().df()["price"])
    date = pd.DataFrame(bql_response.single().df()["DATE"])
    df  = pd.concat([date, px], axis = 1).reset_index(drop=True)
    df.set_index("DATE", inplace = True)
    return df

In [81]:
def xmpt_dvd():
    dps = bq.func.dropna(bq.data.dividend_per_share_12m(dates=bq.func.range('-5y', '0d')))
    bql_universe = 'XMPT US Equity'
    bql_request = bql.Request(bql_universe, {'dps_12m':dps})
    bql_response = bq.execute(bql_request)
    dps = bql_response.single().df()["dps_12m"]
    date = bql_response.single().df()["DATE"]
    df  = pd.concat([date, dps], axis = 1).reset_index(drop=True)
    df.set_index("DATE", inplace = True)
    return df

In [82]:
xmpt_df = xmpt_px().merge(xmpt_dvd(), how = "inner",left_index = True, right_index = True)
xmpt_df["ytm"] = xmpt_df["dps_12m"]/xmpt_df["price"]
xmpt_df.pop("dps_12m")
multi_xmpt = 100 / xmpt_df["price"][0]
xmpt_df["price"] = xmpt_df["price"]*multi_xmpt

In [83]:
fund_list = ["HFRO", "AIF", "BGX", "PHD", "AFT", "ARDC", "JQC", "BGB", "JSD", "NSL", "JRO", "EVF", "FRA", "BGT", "BRW", "JFR", "VVR", "BSL", "EFT", "FCT", "EFR", "EFL", "RSF"]

In [84]:
def ll_px():
    bql_item = bq.func.dropna(bq.func.rolling(bq.func.sum(bq.func.group(bq.data.px_last())), iterationdates=bq.func.range("-5y", "0d")))
    bql_universe = ['{} US Equity'.format(t) for t in fund_list]
    bql_request = bql.Request(bql_universe,  {"price":bql_item})
    bql_response = bq.execute(bql_request)
    df_fund_px = bql.combined_df(bql_response).reset_index()
    df_fund_px.set_index("DATE", inplace = True)
    df_fund_px = pd.DataFrame(df_fund_px["price"]/len(fund_list))
    return df_fund_px

In [85]:
def ll_dvd():
    bql_item = bq.func.dropna(bq.func.rolling(bq.func.sum(bq.func.group(bq.data.dividend_per_share_12m())), iterationdates=bq.func.range("-5y", "0d")))
    bql_universe = ['{} US Equity'.format(t) for t in fund_list]
    bql_request = bql.Request(bql_universe,  {"dps_12m":bql_item})
    bql_response = bq.execute(bql_request)
    df_fund_dvd = bql.combined_df(bql_response).reset_index()
    df_fund_dvd.set_index("DATE", inplace = True)
    df_fund_dvd = pd.DataFrame(df_fund_dvd["dps_12m"]/len(fund_list))
    return df_fund_dvd

In [86]:
ll_df = ll_px().merge(ll_dvd(), how = "inner",left_index = True, right_index = True)
ll_df["ytm"] = ll_df["dps_12m"]/ll_df["price"]
ll_df.pop("dps_12m")
multi_ll = 100 / ll_df["price"][0]
ll_df["price"] = ll_df["price"]*multi_ll

In [87]:
def model(btn = None):
    output.clear_output()
    fig = make_subplots(rows = 2, cols = 3 ,specs=[[{"secondary_y": True}, {"secondary_y": True}, {"secondary_y": True}]
                                                  ,[{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": False}]], subplot_titles=("BDC", "XMPT", "LL", "BDC-LL Yields", "BDC-XMPT Yields", "LL-XMPT yields"))
    fig.add_trace(go.Scatter(x = bizd_df.index, y = bizd_df["price"], name = "BDC price"), row=1, col=1,  secondary_y = False)
    fig.add_trace(go.Scatter(x = bizd_df.index, y = bizd_df["ytm"], name = "BDC ytm"), row=1, col=1,  secondary_y = True)
    fig.add_trace(go.Scatter(x = bizd_df.index, y = xmpt_df["price"], name = "XMPT price"), row=1, col=2,  secondary_y = False)
    fig.add_trace(go.Scatter(x = bizd_df.index, y = xmpt_df["ytm"], name = "XMPT ytm"), row=1, col=2,  secondary_y = True)
    fig.add_trace(go.Scatter(x = bizd_df.index, y = ll_df["price"], name = "LL price"), row=1, col=3,  secondary_y = False)
    fig.add_trace(go.Scatter(x = bizd_df.index, y = ll_df["ytm"], name = "LL ytm"), row=1, col=3,  secondary_y = True)
    fig.add_trace(go.Scatter(x = bizd_df.index, y = bizd_df["ytm"] - ll_df["ytm"], name = "BDC-LL Yields Spread"), row=2, col=1,  secondary_y = False)
    fig.add_trace(go.Scatter(x = bizd_df.index, y = bizd_df["ytm"] - xmpt_df["ytm"], name = "BDC-XMPT Yields Spread"), row=2, col=2,  secondary_y = False)
    fig.add_trace(go.Scatter(x = bizd_df.index, y = ll_df["ytm"] - xmpt_df["ytm"], name = "LL-XMPT Yields Spread"), row=2, col=3,  secondary_y = False)


    fig.update_layout(title_text="LL RV data",width=3000,height = 1000)
    with output:
        fig.show()

    

In [88]:
button1 = ipw.Button(description = 'Show', color = "o")
button1.on_click(model)
output = ipw.Output()
display(button1, output)

Button(description='Show', style=ButtonStyle())

Output()